In [1]:
import pandas as pd
import numpy as np

In [38]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [40]:
clicks_df = pd.read_csv('data/clicks.csv',dtype={'advertiser_id':'int32','action_id':'float32','source_id':'int32','country_code':'category',\
                                                 'latitude':'float32','longitude':'float32','carrier_id':'float32','os_minor':'category',\
                                                  'os_major':'category','specs_brand':'category','timeToClick':'float32','ref_type':'category'})

clicks_df['touchX'] = fix_str_float(clicks_df,'touchX')
clicks_df['touchY'] = fix_str_float(clicks_df,'touchY')
clicks_df['created'] = pd.to_datetime(clicks_df['created'])
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
installs_df = pd.read_csv('data/installs.csv', dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','kind':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df.drop(columns=['session_user_agent','ip_address','device_language'], inplace=True)
installs_df['wifi'].astype('bool', inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
                                                                        'ref_type_id':'category','source_id':'category'})

auctions_df['date'] = pd.to_datetime(auctions_df['date'])
print('setup done')

setup done


In [107]:
installs_df.head(5)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,user_agent,event_uuid,kind,wifi
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,NaN,3.739127e+17,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,NaN,7.805539e+18,NaN,NaN,NaN,NaN
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,NaN,8.355496e+18,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,NaN,2.355772e+18,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,NaN,6.156971e+18,NaN,NaN,NaN,NaN


In [102]:
events_df['connection_type'].describe()

count       1809296
unique            4
top       Cable/DSL
freq        1291512
Name: connection_type, dtype: object

In [103]:
events_df['connection_type'].value_counts()

Cable/DSL    1291512
Cellular      517204
Corporate        527
Dialup            53
Name: connection_type, dtype: int64

In [105]:
events_df['connection_type'].isnull().sum()


5935285

In [111]:
datos = pd.merge(auctions_df, installs_df, left_on='device_id', right_on='ref_hash', how='outer')

(52788284, 17)